<a href="https://colab.research.google.com/github/eiziiaizii1/ceng442-assignment1-GroupTAFB/blob/main/Notebook_ceng442_assignment1_GroupTAFB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CENG442 Assignment 1 - Azerbaijani Text Preprocessing & Word Embeddings

**Group Members:**
* Talha Ubeydullah Gamga | 20050111078
* Aziz Önder | 22050141021
* Muhammed Fatih Asan | 23050151026
* Buğra Bildiren | 20050111022

This notebook contains a full pipeline for cleaning, normalizing, and preparing Azerbaijani text data for machine learning, with a special focus on sentiment analysis.

In [ ]:
# RUN THIS CELL INITIALLY, IF YOU ARE RUNNING IN COLAB
!git clone https://github.com/eiziiaizii1/ceng442-assignment1-GroupTAFB.git
%cd ceng442-assignment1-GroupTAFB
!pip install pandas gensim openpyxl regex ftfy scikit-learn

fatal: destination path 'ceng442-assignment1-GroupTAFB' already exists and is not an empty directory.
/content/ceng442-assignment1-GroupTAFB


In [ ]:
import re, html, unicodedata
import pandas as pd
from pathlib import Path

# Try to import 'ftfy' for fixing text encoding
try:
    from ftfy import fix_text
# If 'ftfy' is not installed, create a dummy function
except Exception:
    def fix_text(s): return s

## Core Normalization Helpers

First, we define a function for language-specific lowercasing, as Azerbaijani has unique 'i' and 'I' characters.

In [ ]:
# Azerbaijani-aware lowercase
def lower_az(s: str) -> str:
    # Check if input is a string
    if not isinstance(s, str): return ""
    # Normalize unicode characters
    s = unicodedata.normalize("NFC", s)
    # Convert Turkish 'I' to 'ı' and 'İ' to 'i'
    s = s.replace("I", "ı").replace("İ", "i")
    # Standard lowercase and fix a common issue
    s = s.lower().replace("i̇", "i")
    return s

## Regex Definitions and Mappings

Here we define all the regular expressions (regex) and data maps we will use to find and replace patterns in the text.

In [ ]:
# --- Regex Definitions ---

# Finds HTML tags (e.g., <br>, <strong>)
HTML_TAG_RE = re.compile(r"<[^>]+>")
# Finds URLs (http://... or www....)
URL_RE = re.compile(r"(https?://\S+|www\.\S+)", re.IGNORECASE)
# Finds email addresses
EMAIL_RE = re.compile(r"\b[\w\.-]+@[\w\.-]+\.\w+\b", re.IGNORECASE)
# Finds phone numbers
PHONE_RE = re.compile(r"\+?\d[\d\-\s\(\)]{6,}\d")
# Finds user mentions (like @username)
USER_RE = re.compile(r"@\w+")
# Finds repeated punctuation (e.g., "!!", "???")
MULTI_PUNCT = re.compile(r"([!?.,;:])\1{1,}")
# Finds extra spaces
MULTI_SPACE = re.compile(r"\s+")
# Finds characters repeated 3+ times (e.g., "sooo")
REPEAT_CHARS= re.compile(r"(.)\1{2,}", flags=re.UNICODE)

# The main rule to find (tokenize) words, numbers, or special tags
TOKEN_RE = re.compile(
    r"[A-Za-zƏəĞğIıİiÖöÜüÇçŞşXxQq]+(?:'[A-Za-zƏəĞğIıİiÖöÜüÇçŞşXxQq]+)?"  # Words
    r"|<NUM>|URL|EMAIL|PHONE|USER|EMO(?:POS|NEG)"  # Tags
)

# --- Mappings ---

# Map for converting emojis to tags (EMO_POS, EMO_NEG)
EMO_MAP = {"🙂":"EMO_POS", "😀":"EMO_POS", "😍":"EMO_POS", "😊":"EMO_POS", "👍":"EMO_POS",
           "☹":"EMO_NEG", "🙁":"EMO_NEG", "😠":"EMO_NEG", "😡":"EMO_NEG", "👎":"EMO_NEG"}

# Map for correcting common slangs
SLANG_MAP = {"slm":"salam", "tmm":"tamam", "sagol":"sağol", "cox":"çox", "yaxsi":"yaxşı"}

# Words that indicate negation
NEGATORS = {"yox", "deyil", "heç", "qətiyyən", "yoxdur"}

## Domain-Specific Processing

These functions detect the "domain" (topic) of the text (e.g., News, Reviews) and apply special cleaning rules only for that domain.

In [ ]:
# --- Domain detection ---
# Keywords to detect 'news' domain
NEWS_HINTS = re.compile(r"\b(apa|trend|azertac|reuters|bloomberg|dha|aa)\b", re.I)
# Keywords/symbols for 'social' media domain
SOCIAL_HINTS = re.compile(r"\b(rt)\b|@|#|(?:😂|😍|😊|👍|👎|😡|🙂)")
# Keywords for 'reviews' domain
REV_HINTS = re.compile(r"\b(azn|manat|qiymət|aldım|ulduz|çox yaxşı|çox pis)\b", re.I)

# Function to check text and assign a domain
def detect_domain(text: str) -> str:
    s = text.lower()
    if NEWS_HINTS.search(s): return "news"
    if SOCIAL_HINTS.search(s): return "social"
    if REV_HINTS.search(s): return "reviews"
    return "general"

# --- Domain-specific normalization (reviews) ---
# Finds prices (e.g., "10 azn") for reviews
PRICE_RE = re.compile(r"\b\d+\s*(azn|manat)\b", re.I)
# Finds star ratings (e.g., "5 ulduz") for reviews
STARS_RE = re.compile(r"\b([1-5])\s*ulduz\b", re.I)
# Finds positive phrases for reviews
POS_RATE = re.compile(r"\bçox yaxşı\b")
# Finds negative phrases for reviews
NEG_RATE = re.compile(r"\bçox pis\b")

# Function to apply special rules based on domain
def domain_specific_normalize(cleaned: str, domain: str) -> str:
    # Only apply these rules for 'reviews'
    if domain == "reviews":
        # Replace price with <PRICE> tag
        s = PRICE_RE.sub(" <PRICE> ", cleaned)
        # Replace stars with <STARS_n> tag
        s = STARS_RE.sub(lambda m: f" <STARS_{m.group(1)}> ", s)
        # Replace positive phrase
        s = POS_RATE.sub(" <RATING_POS> ", s)
        # Replace negative phrase
        s = NEG_RATE.sub(" <RATING_NEG> ", s)
        # Clean up extra spaces
        return " ".join(s.split())
    # Return original text if not 'reviews'
    return cleaned

# --- Domain tag token for corpus (no punctuation) ---
# Adds a domain tag (e.g., 'domnews') to the start of a line
def add_domain_tag(line: str, domain: str) -> str:
    return f"dom{domain} " + line # e.g., 'domnews', 'domreviews'

## Main Text Normalization Function

This is the main function that combines all the previous helpers to perform a full text-cleaning pipeline.

In [ ]:
# The main function to clean one string of text
def normalize_text_az(s: str, numbers_to_token=True, keep_sentence_punct=False) -> str:

    if not isinstance(s, str): return ""

    # First, replace all emojis using the map
    for emo, tag in EMO_MAP.items():
        s = s.replace(emo, f" {tag} ")

    # Fix potential text encoding issues
    s = fix_text(s)
    # Convert HTML entities (like &amp;)
    s = html.unescape(s)
    # Remove HTML tags
    s = HTML_TAG_RE.sub(" ", s)
    # Replace URLs with 'URL' token
    s = URL_RE.sub(" URL ", s)
    # Replace emails with 'EMAIL' token
    s = EMAIL_RE.sub(" EMAIL ", s)
    # Replace phones with 'PHONE' token
    s = PHONE_RE.sub(" PHONE ", s)
    # Handle hashtags, try to split camelCase
    s = re.sub(r"#([A-Za-z0-9_]+)", lambda m: " " + re.sub('([a-z])([A-Z])', r'\1 \2', m.group(1)) + " ", s)
    # Replace user mentions with 'USER' token
    s = USER_RE.sub(" USER ", s)
    # Apply the Azeri-specific lowercase
    s = lower_az(s)
    # Reduce repeated punctuation (e.g., "!!" -> "!")
    s = MULTI_PUNCT.sub(r"\1", s)

    # If option is True
    if numbers_to_token:
        # Replace numbers with '<NUM>' token
        s = re.sub(r"\d+", " <NUM> ", s)

    # If keeping sentence endings
    if keep_sentence_punct:
        # Remove all symbols *except* sentence punctuation
        s = re.sub(r"[^\w\s<>'əğıöşüçƏĞIİÖŞÜÇxqXQ.!?]", " ", s)
    else:
        # Remove all symbols
        s = re.sub(r"[^\w\s<>'əğıöşüçƏĞIİÖŞÜÇxqXQ]", " ", s)

    # Fix extra spaces
    s = MULTI_SPACE.sub(" ", s).strip()

    # Split the clean string into a list of tokens
    toks = TOKEN_RE.findall(s)

    norm = []
    mark_neg = 0 # Counter for negation

    # Loop through each token
    for t in toks:
        # Reduce repeated chars (e.g., "goood" -> "good")
        t = REPEAT_CHARS.sub(r"\1\1", t)
        # Correct slang if found in the map
        t = SLANG_MAP.get(t, t)

        # If this token is a negation word
        if t in NEGATORS:
            norm.append(t);
            # Mark the next 3 words as negative
            mark_neg = 3;
            continue

        # If a word is marked
        if mark_neg > 0 and t not in {"URL", "EMAIL", "PHONE", "USER"}:
            # Add a '_NEG' suffix
            norm.append(t + "_NEG");
            mark_neg -= 1
        else:
            norm.append(t)

    # Final small clean-up (remove single letters)
    norm = [t for t in norm if not (len(t) == 1 and t not in {"o", "e", "ə"})]
    # Join tokens back into a string
    return " ".join(norm).strip()

## Sentiment Value Mapping

This function converts different types of labels (e.g., "positive", "1", "mənfi") into a single, standard numeric format (0.0, 0.5, 1.0).

In [ ]:
# Function to convert different labels to a standard number
def map_sentiment_value(v, scheme: str):

    # For binary (0/1) classification
    if scheme == "binary":
        try: return 1.0 if int(v) == 1 else 0.0
        except Exception: return None

    s = str(v).strip().lower()

    # Check for positive labels
    if s in {"pos", "positive", "1", "müsbət", "pozitiv", "good"}: return 1.0
    # Check for neutral labels
    if s in {"neu", "neutral", "2", "neytral"}: return 0.5
    # Check for negative labels
    if s in {"neg", "negative", "0", "mənfi", "neqativ", "bad"}: return 0.0

    return None

## Main File Processing Function

This final function reads an Excel file, applies all the cleaning and mapping functions to the correct columns, and saves the result as a new two-column (text, sentiment) Excel file.

In [ ]:
# The main function to process an entire Excel file
def process_file(in_path, text_col, label_col, scheme, out_two_col_path,
                 remove_stopwords=False):

    # Read the input Excel file
    df = pd.read_excel(in_path)

    # Remove extra columns if they exist
    for c in ["Unnamed: 0", "index"]:
        if c in df.columns: df = df.drop(columns=[c])

    # Check that required columns exist
    assert text_col in df.columns and label_col in df.columns, f"Missing columns in {in_path}"

    # --- Data Cleaning Steps ---
    # Remove rows with no text
    df = df.dropna(subset=[text_col])
    # Remove rows with empty text
    df = df[df[text_col].astype(str).str.strip().str.len() > 0]
    # Remove duplicate rows
    df = df.drop_duplicates(subset=[text_col])

    # Create 'cleaned_text' column by applying main normalizer
    df["cleaned_text"] = df[text_col].astype(str).apply(lambda s: normalize_text_az(s))

    # Detect the domain for each text
    df["__domain__"] = df[text_col].astype(str).apply(detect_domain)
    # Apply domain-specific normalization *after* general cleaning
    df["cleaned_text"] = df.apply(lambda r:
                                domain_specific_normalize(r["cleaned_text"], r["__domain__"]), axis=1)

    # If stopwatch removal is enabled
    if remove_stopwords:
        # Define the set of stopwords
        sw = set(["və", "ilə", "amma", "ancaq", "lakin", "ya", "həm", "artıq", "çox", "heç",
                  "qətiyyən", "ki", "bu", "bir", "o", "biz", "siz", "sən", "mən", "az", "ən",
                  "orada", "burada", "bütün", "hər", "də", "da", "üçün"])
        # Make sure *not* to remove sentiment-related words
        for keep in ["deyil", "yox", "yoxdur"]:
            sw.discard(keep)
        # Apply the stopword removal
        df["cleaned_text"] = df["cleaned_text"].apply(lambda s: " ".join([t for t in
                                                               s.split() if t not in sw]))

    # Create 'sentiment_value' column by mapping labels
    df["sentiment_value"] = df[label_col].apply(lambda v: map_sentiment_value(v, scheme))
    # Remove rows where sentiment could not be mapped
    df = df.dropna(subset=["sentiment_value"])
    df["sentiment_value"] = df["sentiment_value"].astype(float)

    # --- Save Output ---
    # Select only the final two columns
    out_df = df[["cleaned_text", "sentiment_value"]].reset_index(drop=True)

    # Create the output directory if it doesn't exist
    Path(out_two_col_path).parent.mkdir(parents=True, exist_ok=True)
    # Save the final data to a new Excel file
    out_df.to_excel(out_two_col_path, index=False)

    # Print a confirmation message
    print(f"Saved: {out_two_col_path} (rows={len(out_df)})")

## Corpus Building Function

This function reads all input files, normalizes text into sentences, adds domain tags (e.g., domnews), and saves them to a single corpus_all.txt file for model training.

In [ ]:
def build_corpus_txt(input_files, text_cols, out_txt="corpus_all.txt"):
    """Create domain-tagged, lowercase, punctuation-free corpus (one sentence per
    line).
    """
    lines = []
    for (f, text_col) in zip(input_files, text_cols):
        df = pd.read_excel(f)
        for raw in df[text_col].dropna().astype(str):
            dom = detect_domain(raw)
            # Normalize text, but keep sentence punctuation for splitting
            s = normalize_text_az(raw, keep_sentence_punct=True)

            # Split text into sentences
            parts = re.split(r"[.!?]+", s)
            for p in parts:
                p = p.strip()
                if not p: continue
                # Remove remaining punctuation
                p = re.sub(r"[^\w\səğıöşüçƏĞIİÖŞÜÇxqXQ]", " ", p)
                p = " ".join(p.split()).lower()
                if p:
                    lines.append(f"dom{dom} " + p)

    with open(out_txt, "w", encoding="utf-8") as w:
        for ln in lines:
            w.write(ln + "\n")
    print(f"Wrote {out_txt} with {len(lines)} lines")

## Main Execution Block

This block defines the input file configuration (CFG) and runs the main pipeline: processing each file into a _2col.xlsx output, and then building the combined corpus_all.txt.

In [ ]:
if __name__ == "__main__":
    CFG = [
        # Using the exact filenames you provided:
        ("data/labeled-sentiment.xlsx", "text", "sentiment", "tri"),
        ("data/merged_dataset_CSV__1_.xlsx", "text", "labels", "binary"),
        ("data/test__1_.xlsx", "text", "label", "binary"),
        ("data/train__3_.xlsx", "text", "label", "binary"),
        ("data/train-00000-of-00001.xlsx", "text", "labels", "tri"),
    ]

    # 1. Process all files into two-column (text, sentiment) outputs
    print("--- Processing files for 2-column output ---")
    for fname, tcol, lcol, scheme in CFG:
        # Output path now points to the 'clean_data/' folder
        out = f"clean_data/{Path(fname).stem}_2col.xlsx"
        process_file(fname, tcol, lcol, scheme, out, remove_stopwords=False)

    # 2. Build the combined domain-tagged, punctuation-free corpus
    print("\n--- Building combined corpus text file ---")
    build_corpus_txt([c[0] for c in CFG], [c[1] for c in CFG],
                     out_txt="corpus_all.txt")

--- Processing files for 2-column output ---
Saved: clean_data/labeled-sentiment_2col.xlsx (rows=2955)
Saved: clean_data/merged_dataset_CSV__1__2col.xlsx (rows=55662)
Saved: clean_data/test__1__2col.xlsx (rows=4198)
Saved: clean_data/train__3__2col.xlsx (rows=19557)
Saved: clean_data/train-00000-of-00001_2col.xlsx (rows=41756)

--- Building combined corpus text file ---
Wrote corpus_all.txt with 124353 lines


## Training Word2Vec and FastText embedding Models

Following the preprocessing steps, we now have five cleaned Excel files. The next task is to train the **Word2Vec** and **FastText** models as specified in the assignment.

The code below performs the following actions:
1.  Initializes an empty list called `sentences`.
2.  Loops through each of the five `_2col.xlsx` files and reads the `cleaned_text` column using `pandas`.
3.  Converts each row of cleaned text into a list of tokens (by splitting on spaces) and adds these lists to the main `sentences` collection.
4.  Creates the `embeddings/` directory if it doesn't already exist.
5.  Trains a `Word2Vec` model using the `sentences` corpus. Key parameters include `vector_size=300`, `window=5`, `min_count=3`, and `sg=1` (Skip-gram).
6.  Trains a `FastText` model using the same corpus and similar parameters, but also includes subword information (`min_n=3`, `max_n=6`).
7.  Saves both trained models to the `embeddings/` folder as `word2vec.model` and `fasttext.model`.

In [ ]:
from gensim.models import Word2Vec, FastText
import pandas as pd
from pathlib import Path

files = [
    f"./clean_data/labeled-sentiment_2col.xlsx",
    f"./clean_data/test_1_2col.xlsx",
    f"./clean_data/train_3_2col.xlsx",
    f"./clean_data/train-00000-of-00001_2col.xlsx",
    f"./clean_data/merged_dataset_CSV_1_2col.xlsx",
]

sentences = []
for f in files:
    df = pd.read_excel(f, usecols=["cleaned_text"])
    sentences.extend(df["cleaned_text"].astype(str).str.split().tolist())

Path("embeddings").mkdir(exist_ok=True)
w2v = Word2Vec(sentences=sentences, vector_size=300, window=5, min_count=3, sg=1,
negative=10, epochs=10)
w2v.save("embeddings/word2vec.model")
ft  = FastText(sentences=sentences, vector_size=300, window=5, min_count=3, sg=1,
min_n=3, max_n=6, epochs=10)
ft.save("embeddings/fasttext.model")
print("Saved embeddings.")

Saved embeddings.


## Model Evaluation: Word2Vec vs. FastText (Quantitative & Qualitative Metrics)

Model Evaluation section presents a comparative evaluation of the generated Word2Vec and FastText models. To assess their respective strengths in capturing the semantics of the Azerbaijani corpus, the analysis employs three distinct evaluation metrics.

### 1. Lexical Coverage (Quantitative)

This metric quantifies the **vocabulary coverage** of each model, measuring the percentage of unique tokens from our cleaned datasets that are found within the model's learned vocabulary.

This is a critical test for comparing the two architectures. Word2Vec, being a word-level model, is inherently limited to its training vocabulary and cannot represent **out-of-vocabulary (OOV)** words. In contrast, FastText, which learns vectors for character n-grams (subwords), can construct vectors for *any* word, including neologisms, misspellings, or rare words not encountered during training.

### 2. Semantic Similarity (Quantitative)

A successful embedding model should capture meaningful **semantic relationships**, placing words with similar meanings close together in the vector space and words with opposite meanings far apart.

To quantify this, we measure the average **cosine similarity** for two predefined sets of word pairs:
* **Synonym Pairs** (e.g., `yaxşı`, `əla`): We expect a high similarity score (close to 1.0), indicating semantic proximity.
* **Antonym Pairs** (e.g., `yaxşı`, `pis`): We expect a low or negative similarity score (close to -1.0 or 0.0), indicating semantic distance.

A "Separation Score" (calculated as `Synonym Similarity - Antonym Similarity`) is then used to provide a single, robust measure of the model's ability to discriminate between semantic similarity and opposition. A higher separation score is better.

### 3. Nearest Neighbors Analysis (Qualitative)

Beyond quantitative scores, a **qualitative analysis** of the embedding space is performed by inspecting the **nearest neighbors** for a set of predefined seed words.

By examining the top 5 most similar words for a given seed (e.g., `bahalı` or `pis`), we can intuitively assess the quality of the learned representations. This helps us judge whether the model has learned logical contexts (e.g., are the neighbors of "expensive" other price-related words?) or if it has merely learned superficial co-occurrence patterns.

In [ ]:
import pandas as pd
from gensim.models import Word2Vec, FastText
import re

w2v = Word2Vec.load("embeddings/word2vec.model")
ft  = FastText.load("embeddings/fasttext.model")

seed_words = ["yaxşı","pis","çox","bahalı","ucuz","mükəmməl","dəhşət","<PRICE>","<RATING_POS>"]
syn_pairs  = [("yaxşı","əla"), ("bahalı","qiymətli"), ("ucuz","sərfəli")]
ant_pairs  = [("yaxşı","pis"), ("bahalı","ucuz")]

def lexical_coverage(model, tokens):
    vocab = model.wv.key_to_index
    return sum(1 for t in tokens if t in vocab) / max(1,len(tokens))

files = [
    f"./clean_data/labeled-sentiment_2col.xlsx",
    f"./clean_data/test_1_2col.xlsx",
    f"./clean_data/train_3_2col.xlsx",
    f"./clean_data/train-00000-of-00001_2col.xlsx",
    f"./clean_data/merged_dataset_CSV_1_2col.xlsx",
]

def read_tokens(f):
    df = pd.read_excel(f, usecols=["cleaned_text"])
    return [t for row in df["cleaned_text"].astype(str) for t in row.split()]

print("== Lexical coverage (per dataset) ==")
for f in files:
    toks = read_tokens(f)
    cov_w2v = lexical_coverage(w2v, toks)
    cov_ftv = lexical_coverage(ft, toks)  # FT still embeds OOV via subwords
    print(f"{f}: W2V={cov_w2v:.3f}, FT(vocab)={cov_ftv:.3f}")

from numpy import dot
from numpy.linalg import norm

def cos(a,b): return float(dot(a,b)/(norm(a)*norm(b)))

def pair_sim(model, pairs):
    vals = []
    for a,b in pairs:
        try: vals.append(model.wv.similarity(a,b))
        except KeyError: pass
    return sum(vals)/len(vals) if vals else float('nan')

syn_w2v = pair_sim(w2v, syn_pairs)
syn_ft  = pair_sim(ft,  syn_pairs)
ant_w2v = pair_sim(w2v, ant_pairs)
ant_ft  = pair_sim(ft,  ant_pairs)

print("\n== Similarity (higher better for synonyms; lower better for antonyms) ==")
print(f"Synonyms: W2V={syn_w2v:.3f}, FT={syn_ft:.3f}")
print(f"Antonyms: W2V={ant_w2v:.3f}, FT={ant_ft:.3f}")
print(f"Separation (Syn - Ant): W2V={(syn_w2v - ant_w2v):.3f}, FT={(syn_ft - ant_ft):.3f}")

def neighbors(model, word, k=5):
  try: return [w for w,_ in model.wv.most_similar(word, topn=k)]
  except KeyError: return []

print("\n== Nearest neighbors (qualitative) ==")
for w in seed_words:
  print(f"  W2V NN for '{w}':", neighbors(w2v, w))
  print(f"  FT  NN for '{w}':", neighbors(ft,  w))

# (Optional) domain drift if you train domain-specific models separately:
# drift(word, model_a, model_b) = 1 - cos(vec_a, vec_b)

== Lexical coverage (per dataset) ==
./clean_data/labeled-sentiment_2col.xlsx: W2V=0.930, FT(vocab)=0.930
./clean_data/test_1_2col.xlsx: W2V=0.915, FT(vocab)=0.915
./clean_data/train_3_2col.xlsx: W2V=0.919, FT(vocab)=0.919
./clean_data/train-00000-of-00001_2col.xlsx: W2V=0.923, FT(vocab)=0.923
./clean_data/merged_dataset_CSV_1_2col.xlsx: W2V=0.899, FT(vocab)=0.899

== Similarity (higher better for synonyms; lower better for antonyms) ==
Synonyms: W2V=0.361, FT=0.465
Antonyms: W2V=0.335, FT=0.424
Separation (Syn - Ant): W2V=0.027, FT=0.040

== Nearest neighbors (qualitative) ==
  W2V NN for 'yaxşı': ['<RATING_POS>', 'yaxshi', 'iyi', 'yaxşı.', 'olar.']
  FT  NN for 'yaxşı': ['yaxşı!', 'yaxşıı', 'yaxşı.', 'yaxşıkı', ',yaxşı']
  W2V NN for 'pis': ['<RATING_NEG>', '<STARS_LOW>', 'pis.', 'pisdir', 'pisdi']
  FT  NN for 'pis': ['pis!', 'pis,', 'pis.', 'pis.pul', 'piis']
  W2V NN for 'çox': ['çoox', 'çöx', 'tətbiqidir', 'əladir', 'elverisli']
  FT  NN for 'çox': ['çoxçox', 'çox.çox', '(çox', '

In [ ]:
# Not quite sure about its correctness,
# but as the vocab sizes differs significantly, it's possible to observe inconsistent values

# Domain Drift Analysis

from gensim.models import Word2Vec
import pandas as pd
from numpy import dot
from numpy.linalg import norm
import random

def cos_sim(v1, v2):
    return dot(v1, v2) / (norm(v1) * norm(v2))

def domain_drift(word, model_a, model_b):
    try:
        vec_a = model_a.wv[word]
        vec_b = model_b.wv[word]
        return 1 - cos_sim(vec_a, vec_b)
    except KeyError:
        return None

# Read the domain-tagged corpus
domain_sentences = {"news": [], "social": [], "reviews": [], "general": []}

with open("corpus_all.txt", "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        if line.startswith("dom"):
            parts = line.split(maxsplit=1)
            if len(parts) == 2:
                domain_tag = parts[0].replace("dom", "")
                text = parts[1]
                if domain_tag in domain_sentences:
                    domain_sentences[domain_tag].append(text.split())

# Filter the domans based on the sentences numbers
min_samples = 2000
valid_domains = {k: v for k, v in domain_sentences.items() if len(v) >= min_samples}

print(f"=== Domain Statistics ===")
for dom, sents in domain_sentences.items():
    print(f"{dom:10s}: {len(sents):6d} sentences")

if len(valid_domains) < 2:
    print(f"\nNeed at least 2 domains with ≥{min_samples} sentences for comparison.")
else:
    # Find the smallest domain
    sample_size = min(len(sents) for sents in valid_domains.values())
    print(f"\n=== Balancing: using {sample_size} sentences per domain ===")

    # take equal numbered samples from each domain
    #random.seed(42)
    balanced_domains = {}
    for dom, sents in valid_domains.items():
        balanced_domains[dom] = random.sample(sents, sample_size)

    # train the models
    domain_models = {}
    print("\n=== Training balanced Word2Vec models ===")
    for dom, sents in balanced_domains.items():
        model = Word2Vec(sentences=sents, vector_size=200, window=5,
                        min_count=3, sg=1, epochs=10, seed=42)
        domain_models[dom] = model
        print(f"Trained {dom}: {len(sents)} sentences, vocab={len(model.wv)}")

    # Domain drift analysis
    test_words = ["yaxşı", "pis", "çox", "yaxşıdır", "pisdir", "mükəmməl",
                  "telefon", "film", "məhsul"]

    print("\n=== Domain Drift Analysis (Balanced Data) ===")
    print("Lower = similar context, Higher = different context\n")

    domains = list(domain_models.keys())
    for i, dom_a in enumerate(domains):
        for dom_b in domains[i+1:]:
            print(f"\n{dom_a.upper()} ↔ {dom_b.upper()}:")
            print("-" * 50)

            drifts = []
            for w in test_words:
                drift = domain_drift(w, domain_models[dom_a], domain_models[dom_b])
                if drift is not None:
                    drifts.append((w, drift))

            if drifts:
                drifts.sort(key=lambda x: x[1])
                for w, d in drifts:
                    status = "✓ stable" if d < 0.3 else "⚠ drift" if d < 0.6 else "✗ different"
                    print(f"  {w:15s} → {d:.3f}  {status}")
            else:
                print("  (no common words)")